In [37]:
import pandas as pd
import math

In [38]:
ocpf_df = pd.read_csv('../original_data/ocpf-output.csv')
ocpf_df = ocpf_df[ocpf_df.State.isin(['MA'])][['Date', 'City', 'Amount']]
ocpf_df['Amount'] = ocpf_df['Amount'].replace('[\$]','', regex=True).astype(float)

ocpf_df = ocpf_df.groupby(['City'])['Amount'].apply(sum).reset_index()

# ocpf_df.to_csv("ocpf_summaries.csv")

ocpf_df['City'] = ocpf_df['City'].map(lambda x: str(x).lower())
print(ocpf_df)


                 City    Amount
0             1/30/16    620.00
1             amherst    989.08
2             andover    750.00
3            abington   2682.65
4               acton   5196.62
5            acushnet    525.00
6              agawam   1478.79
7             allston   4500.00
8            amesbury   3750.00
9             amherst   4294.93
10            andover  17050.69
11          arlington  11449.00
12         ashburnham    600.00
13            ashland   2991.13
14              athol    800.00
15          attleboro    900.00
16             auburn   9542.68
17         auburndale   3695.31
18               avon   1500.00
19             boston   2217.00
20           boylston    750.00
21         barnstable   2403.11
22         barrington    750.00
23            bedford   5463.00
24        belchertown   1985.60
25         bellingham    597.72
26            belmont   3689.20
27        bernardston    551.05
28            beverly   6789.00
29      beverly farms    750.00
..      

In [39]:
#################### 
# Senate districts #
####################


senate_df = pd.read_csv("../cleaned_data/senate_districts_mapped_final.csv", names=['city_town', 'senate_district', 'n/a'], header=None)[['city_town', 'senate_district']]

senate_df['city_town'] = senate_df['city_town'].apply(lambda x: str(x).split(" -")[0]).map(lambda x: x.lower())
senate_df['city_town'] = senate_df['city_town'].replace('[\:]','', regex=True).str.strip()
senate_df['senate_district'] = senate_df['senate_district'].replace('(and)','&', regex=True)

print(senate_df)


            city_town                            senate_district
0               adams   Berkshire, Hampshire, Franklin & Hampden
1              alford   Berkshire, Hampshire, Franklin & Hampden
2              becket   Berkshire, Hampshire, Franklin & Hampden
3            cheshire   Berkshire, Hampshire, Franklin & Hampden
4          clarksburg   Berkshire, Hampshire, Franklin & Hampden
5              dalton   Berkshire, Hampshire, Franklin & Hampden
6            egremont   Berkshire, Hampshire, Franklin & Hampden
7             florida   Berkshire, Hampshire, Franklin & Hampden
8    great barrington   Berkshire, Hampshire, Franklin & Hampden
9             hancock   Berkshire, Hampshire, Franklin & Hampden
10           hinsdale   Berkshire, Hampshire, Franklin & Hampden
11       lanesborough   Berkshire, Hampshire, Franklin & Hampden
12                lee   Berkshire, Hampshire, Franklin & Hampden
13              lenox   Berkshire, Hampshire, Franklin & Hampden
14           monterey   B

In [40]:
senate_df = senate_df.groupby(['city_town'])['senate_district'].apply(list).reset_index()
print(senate_df)
ocpf_df = ocpf_df.groupby(['City'])['Amount'].apply(sum).reset_index()
print(ocpf_df)

            city_town                                    senate_district
0            abington                               [Norfolk & Plymouth]
1               acton                            [Middlesex & Worcester]
2            acushnet                        [Second Bristol & Plymouth]
3               adams         [Berkshire, Hampshire, Franklin & Hampden]
4              agawam                       [Second Hampden & Hampshire]
5              alford         [Berkshire, Hampshire, Franklin & Hampden]
6            amesbury                                      [First Essex]
7             amherst                  [Hampshire, Franklin & Worcester]
8             andover                         [Second Essex & Middlesex]
9            aquinnah                                     [Cape & Isl&s]
10          arlington                                 [Fourth Middlesex]
11         ashburnham        [Worcester, Hampden, Hampshire & Middlesex]
12              ashby        [Worcester, Hampden, H

In [41]:

senate_ocpf = pd.concat([ocpf_df.set_index('City'), senate_df.set_index('city_town')], axis=1, join='inner')
senate_ocpf.index.name = 'city_town'
print(senate_ocpf)
senate_ocpf.to_csv("../cleaned_data/contribution_data_senate_districts.csv")


                     Amount                                    senate_district
city_town                                                                     
abington            2682.65                               [Norfolk & Plymouth]
acton               5196.62                            [Middlesex & Worcester]
acushnet             525.00                        [Second Bristol & Plymouth]
agawam              1478.79                       [Second Hampden & Hampshire]
amesbury            3750.00                                      [First Essex]
amherst             5284.01                  [Hampshire, Franklin & Worcester]
andover            17800.69                         [Second Essex & Middlesex]
arlington          11449.00                                 [Fourth Middlesex]
ashburnham           600.00        [Worcester, Hampden, Hampshire & Middlesex]
ashland             2991.13                       [Second Middlesex & Norfolk]
athol                800.00        [Worcester, Hampd

In [42]:
#################### 
# House districts #
####################

house_df = pd.read_csv("../cleaned_data/house_districts_mapped_final.csv", names=['city_town', 'house_district', 'n/a'], header=None)[['city_town', 'house_district']]

house_df['city_town'] = house_df['city_town'].apply(lambda x: str(x).split(" -")[0]).apply(lambda x: str(x).split(" (")[0]).map(lambda x: x.lower())
house_df['city_town'] = house_df['city_town'].replace('[\:]','', regex=True).str.strip()

print(house_df)

         city_town                   house_district
0         brewster                 First Barnstable
1           dennis                 First Barnstable
2         yarmouth                 First Barnstable
3       barnstable                Second Barnstable
4         yarmouth                Second Barnstable
5           bourne                 Third Barnstable
6         falmouth                 Third Barnstable
7          mashpee                 Third Barnstable
8         brewster                Fourth Barnstable
9          chatham                Fourth Barnstable
10         eastham                Fourth Barnstable
11         harwich                Fourth Barnstable
12         orleans                Fourth Barnstable
13    provincetown                Fourth Barnstable
14           truro                Fourth Barnstable
15       wellfleet                Fourth Barnstable
16      barnstable                 Fifth Barnstable
17          bourne                 Fifth Barnstable
18        sa

In [43]:
house_districts = list(house_df['house_district'])
ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(math.floor(n/10)%10!=1)*(n%10<4)*n%10::4])
dumb_list = [ordinal(n) for n in range(0,38)]

for i in range(len(house_districts)):
    splitter = house_districts[i].split(" ")
    if splitter[0] == 'Barnstable,':
        continue
    if splitter[0] == 'First':
        splitter[0] = dumb_list[1]
    elif splitter[0] == 'Second':
        splitter[0] = dumb_list[2]
    elif splitter[0] == 'Third':
        splitter[0] = dumb_list[3]
    elif splitter[0] == 'Fourth':
        splitter[0] = dumb_list[4]
    elif splitter[0] == 'Fifth':
        splitter[0] = dumb_list[5]
    elif splitter[0] == 'Sixth':
        splitter[0] = dumb_list[6]
    elif splitter[0] == 'Seventh':
        splitter[0] = dumb_list[7]
    elif splitter[0] == 'Eighth':
        splitter[0] = dumb_list[8]
    elif splitter[0] == 'Eigth':
        splitter[0] = dumb_list[8]
    elif splitter[0] == 'Ninth':
        splitter[0] = dumb_list[9]
    elif splitter[0] == 'Tenth':
        splitter[0] = dumb_list[10]
    elif splitter[0] == 'Eleventh':
        splitter[0] = dumb_list[11]
    elif splitter[0] == 'Twelfth':
        splitter[0] = dumb_list[12]
    elif splitter[0] == 'Thirteenth':
        splitter[0] = dumb_list[13]
    elif splitter[0] == 'Fourteenth':
        splitter[0] = dumb_list[14]
    elif splitter[0] == 'Fifteenth':
        splitter[0] = dumb_list[15]
    elif splitter[0] == 'Sixteenth':
        splitter[0] = dumb_list[16]
    elif splitter[0] == 'Seventeenth':
        splitter[0] = dumb_list[17]
    elif splitter[0] == 'Eighteenth':
        splitter[0] = dumb_list[18]
    elif splitter[0] == 'Nineteenth':
        splitter[0] = dumb_list[19]
    elif splitter[0] == 'Ninteenth':
        splitter[0] = dumb_list[19]
    elif splitter[0] == 'Twentieth':
        splitter[0] = dumb_list[20]
    elif splitter[0] == 'Twenty-First':
        splitter[0] = dumb_list[21]
    elif splitter[0] == 'Twenty-Second':
        splitter[0] = dumb_list[22]
    elif splitter[0] == 'Twenty-Third':
        splitter[0] = dumb_list[23]
    elif splitter[0] == 'Twenty-Fourth':
        splitter[0] = dumb_list[24]
    elif splitter[0] == 'Twenty-Fifth':
        splitter[0] = dumb_list[25]
    elif splitter[0] == 'Twenty-Sixth':
        splitter[0] = dumb_list[26]
    elif splitter[0] == 'Twenty-Seventh':
        splitter[0] = dumb_list[27]
    elif splitter[0] == 'Twenty-Eighth':
        splitter[0] = dumb_list[28]
    elif splitter[0] == 'Twenty-Ninth':
        splitter[0] = dumb_list[29]
    elif splitter[0] == 'Thirtieth':
        splitter[0] = dumb_list[30]
    elif splitter[0] == 'Thirty-First':
        splitter[0] = dumb_list[31]
    elif splitter[0] == 'Thirty-Second':
        splitter[0] = dumb_list[32]
    elif splitter[0] == 'Thirty-Third':
        splitter[0] = dumb_list[33]
    elif splitter[0] == 'Thirty-Fourth':
        splitter[0] = dumb_list[34]
    elif splitter[0] == 'Thirty-Fifth':
        splitter[0] = dumb_list[35]
    elif splitter[0] == 'Thirty-Sixth':
        splitter[0] = dumb_list[36]
    elif splitter[0] == 'Thirty-Seventh':
        splitter[0] = dumb_list[37]

        
    district = ''
    for j in range(len(splitter)):
        if j != len(splitter)-1:
            district += splitter[j]+" "
        else:
            district += splitter[j]
    house_districts[i] = district
    
house_df['house_district'] = house_districts
print(house_df)

         city_town                   house_district
0         brewster                   1st Barnstable
1           dennis                   1st Barnstable
2         yarmouth                   1st Barnstable
3       barnstable                   2nd Barnstable
4         yarmouth                   2nd Barnstable
5           bourne                   3rd Barnstable
6         falmouth                   3rd Barnstable
7          mashpee                   3rd Barnstable
8         brewster                   4th Barnstable
9          chatham                   4th Barnstable
10         eastham                   4th Barnstable
11         harwich                   4th Barnstable
12         orleans                   4th Barnstable
13    provincetown                   4th Barnstable
14           truro                   4th Barnstable
15       wellfleet                   4th Barnstable
16      barnstable                   5th Barnstable
17          bourne                   5th Barnstable
18        sa

In [44]:
house_df = house_df.groupby(['city_town'])['house_district'].apply(list).reset_index()
print(house_df)

            city_town                                     house_district
0            abington                                     [7th Plymouth]
1               acton                   [14th Middlesex, 37th Middlesex]
2            acushnet                                     [11th Bristol]
3               adams                                    [1st Berkshire]
4              agawam                                      [3rd Hampden]
5              alford                                    [4th Berkshire]
6            amesbury                                        [1st Essex]
7             amherst                                    [3rd Hampshire]
8             andover                           [17th Essex, 18th Essex]
9            aquinnah                  [Barnstable, Dukes and Nantucket]
10          arlington                   [23rd Middlesex, 24th Middlesex]
11         ashburnham                                    [2nd Worcester]
12              ashby                              

In [45]:
house_ocpf = pd.concat([ocpf_df.set_index('City'), house_df.set_index('city_town')], axis=1, join='inner')
house_ocpf.index.name = 'city_town'
print(house_ocpf)
house_ocpf.to_csv("../cleaned_data/contribution_data_house_districts.csv")



                     Amount                                     house_district
city_town                                                                     
abington            2682.65                                     [7th Plymouth]
acton               5196.62                   [14th Middlesex, 37th Middlesex]
acushnet             525.00                                     [11th Bristol]
agawam              1478.79                                      [3rd Hampden]
amesbury            3750.00                                        [1st Essex]
amherst             5284.01                                    [3rd Hampshire]
andover            17800.69                           [17th Essex, 18th Essex]
arlington          11449.00                   [23rd Middlesex, 24th Middlesex]
ashburnham           600.00                                    [2nd Worcester]
ashland             2991.13                                    [7th Middlesex]
athol                800.00                         

In [46]:
senate_df.to_csv("../cleaned_data/senate_districts_mapped_to_lists.csv")
house_df.to_csv("../cleaned_data/house_districts_mapped_to_lists.csv")